<a href="https://colab.research.google.com/github/wafaabek/Big_data/blob/main/SparkSql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark


In [ ]:
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql.functions import *
from pyspark.sql import Row



Télécharger le fichier :

In [ ]:
!wget https://storage.googleapis.com/books/ngrams/books/googlebooks-eng-all-2gram-20120701-0.gz


--2024-10-22 09:54:28--  https://storage.googleapis.com/books/ngrams/books/googlebooks-eng-all-2gram-20120701-0.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.117.207, 74.125.20.207, 108.177.98.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.117.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 319142622 (304M) [binary/octet-stream]
Saving to: ‘googlebooks-eng-all-2gram-20120701-0.gz.1’

googlebooks-eng-all 100%[===================>] 304.36M   157MB/s    in 1.9s    

2024-10-22 09:54:30 (157 MB/s) - ‘googlebooks-eng-all-2gram-20120701-0.gz.1’ saved [319142622/319142622]



In [ ]:
!gunzip googlebooks-eng-all-2gram-20120701-0.gz


Étape 3: Importer les bibliothèques et créer le DataFrame

In [ ]:
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql.functions import *
from pyspark.sql import Row



In [ ]:


# Initialisation de Spark
conf = SparkConf().setAppName("Ngram Analysis")
sc = SparkContext.getOrCreate(conf=conf)
sqlContext = SQLContext(sc)

# Charger le fichier décompressé
lines = sc.textFile("/content/ngram.csv")

# Diviser les lignes en colonnes : ngram, year, match_count, volume_count
parts = lines.map(lambda line: line.split("\t"))
ngrams = parts.map(lambda p: Row(ngram=p[0], year=int(p[1]), ngramcount=int(p[2]),pages=int(p[3]), books=int(p[4])))

# Créer un DataFrame à partir des lignes
schemaNGrams = sqlContext.createDataFrame(ngrams)

# Afficher un aperçu du DataFrame
schemaNGrams.show(5)



+--------+----+----------+-----+-----+
|   ngram|year|ngramcount|pages|books|
+--------+----+----------+-----+-----+
|! $17.95|1985|         1|    1|    1|
|! $17.95|1987|         1|    1|    1|
|! $17.95|1990|         1|    1|    1|
|! $17.95|1991|         1|    1|    1|
|! $17.95|1992|         5|    5|    5|
+--------+----+----------+-----+-----+
only showing top 5 rows



Étape 4: Enregistrer le DataFrame comme table temporaire

In [ ]:
schemaNGrams.createOrReplaceTempView("ngrams")


**1-Retourner tous les bi-grammes dont le nombre Count est supérieur à cinq.**

Étape 5: Répondre aux questions du TP (Avec SQL :)





In [ ]:
result = sqlContext.sql("SELECT ngram FROM ngrams WHERE ngramcount > 5")
result.show()


+--------+
|   ngram|
+--------+
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
+--------+



Avec l’API SparkSQL :



In [ ]:

result = schemaNGrams.filter(schemaNGrams['ngramcount'] > 5).select('ngram')
result.show()

+--------+
|   ngram|
+--------+
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
+--------+



**Retourner le nombre total de bi-grammes dans chaque année.**

le langage SQL:

In [ ]:
result = sqlContext.sql("SELECT year, COUNT(*) AS cnt FROM ngrams GROUP BY year")
result.show()

+----+---+
|year|cnt|
+----+---+
|1806|  1|
|1840|  1|
|1882|  1|
|2007|  2|
|1887|  1|
|1991|  2|
|1858|  1|
|1833|  1|
|1831|  1|
|1987|  2|
|1999|  2|
|1997|  2|
|1884|  1|
|1845|  1|
|1877|  1|
|1829|  1|
|1867|  1|
|2006|  2|
|1876|  1|
|1879|  1|
+----+---+
only showing top 20 rows



Api spark

In [ ]:
result = schemaNGrams.groupBy("year").count()
result.show()


+----+-----+
|year|count|
+----+-----+
|1806|    1|
|1840|    1|
|1882|    1|
|2007|    2|
|1887|    1|
|1991|    2|
|1858|    1|
|1833|    1|
|1831|    1|
|1987|    2|
|1999|    2|
|1997|    2|
|1884|    1|
|1845|    1|
|1877|    1|
|1829|    1|
|1867|    1|
|2006|    2|
|1876|    1|
|1879|    1|
+----+-----+
only showing top 20 rows



Retourner les bi-grammes qui ont le plus grand nombre de count dans chaque
année.


SQL

In [ ]:
result = sqlContext.sql("SELECT ngram, year, MAX(ngramcount) AS maxCount FROM ngrams GROUP BY ngram, year")
result.show()


+--------+----+--------+
|   ngram|year|maxCount|
+--------+----+--------+
|    ! 09|1880|       2|
|! $17.95|2008|       2|
|    ! 09|1845|       1|
|! $17.95|1995|       1|
|! $17.95|2004|      14|
|    ! 09|1780|       1|
|    ! 09|1833|       1|
|! $17.95|2007|       2|
|    ! 09|1872|       1|
|    ! 09|1861|       1|
|    ! 09|1806|       1|
|    ! 09|1882|       2|
|! $17.95|2001|       5|
|    ! 09|1881|       3|
|    ! 09|1884|       5|
|    ! 09|1823|       1|
|    ! 09|1848|       1|
|    ! 09|1887|       1|
|    ! 09|1803|       1|
|    ! 09|1877|       2|
+--------+----+--------+
only showing top 20 rows



In [ ]:
result = schemaNGrams.groupBy("ngram", "year").max("ngramcount")
result.show()


+--------+----+---------------+
|   ngram|year|max(ngramcount)|
+--------+----+---------------+
|    ! 09|1880|              2|
|! $17.95|2008|              2|
|    ! 09|1845|              1|
|! $17.95|1995|              1|
|! $17.95|2004|             14|
|    ! 09|1780|              1|
|    ! 09|1833|              1|
|! $17.95|2007|              2|
|    ! 09|1872|              1|
|    ! 09|1861|              1|
|    ! 09|1806|              1|
|    ! 09|1882|              2|
|! $17.95|2001|              5|
|    ! 09|1881|              3|
|    ! 09|1884|              5|
|    ! 09|1823|              1|
|    ! 09|1848|              1|
|    ! 09|1887|              1|
|    ! 09|1803|              1|
|    ! 09|1877|              2|
+--------+----+---------------+
only showing top 20 rows



Retourner les bi-grammes contenant ! dans la première partie et 9 dans la **deuxième** **texte en gras**

SQL

In [ ]:
sqlContext.sql("SELECT DISTINCT ngram FROM ngrams WHERE ngram LIKE '%!% %' AND ngram LIKE '% %9%'").show()


+--------+
|   ngram|
+--------+
|    ! 09|
|! $17.95|
+--------+



In [ ]:
schemaNGrams.filter((col("ngram").like("%!% %")) & (col("ngram").like("% %9%"))).select("ngram").distinct().show()


+--------+
|   ngram|
+--------+
|    ! 09|
|! $17.95|
+--------+



Retourner les bi-grammes apparus dans toutes les années présentes dans les données
Avec SQL :

In [ ]:
sqlContext.sql("SELECT ngram FROM ngrams GROUP BY ngram HAVING COUNT(DISTINCT year) = (SELECT COUNT(DISTINCT year) FROM ngrams)").show()


+-----+
|ngram|
+-----+
| ! 09|
+-----+



In [ ]:
distinct_years = schemaNGrams.select("year").distinct().count()

result = schemaNGrams.groupBy("ngram").agg(countDistinct("year").alias("distinct_years")) \
    .filter(f"distinct_years = {distinct_years}") \
    .select("ngram").show()


+-----+
|ngram|
+-----+
| ! 09|
+-----+



Retourner le nombre total de pages et de livres dans lesquels chaque bigramme apparaît pour chaque année disponible, trié par ordre alphabétique.

In [ ]:
sqlContext.sql("SELECT year, ngram, SUM(ngramcount) AS TotalPages, SUM(books) AS TotalBooks FROM ngrams GROUP BY year, ngram ORDER BY ngram ASC").show()


+----+--------+----------+----------+
|year|   ngram|TotalPages|TotalBooks|
+----+--------+----------+----------+
|1998|! $17.95|         4|         3|
|1992|! $17.95|         5|         5|
|1995|! $17.95|         1|         1|
|1997|! $17.95|         6|         5|
|1993|! $17.95|         2|         2|
|1987|! $17.95|         1|         1|
|1985|! $17.95|         1|         1|
|1991|! $17.95|         1|         1|
|1990|! $17.95|         1|         1|
|2007|! $17.95|         2|         2|
|2004|! $17.95|        14|        14|
|1996|! $17.95|         4|         2|
|2005|! $17.95|        13|        13|
|2001|! $17.95|         5|         4|
|2003|! $17.95|         2|         2|
|1999|! $17.95|        11|        10|
|2006|! $17.95|         5|         5|
|2008|! $17.95|         2|         2|
|2000|! $17.95|        11|         9|
|2002|! $17.95|         5|         5|
+----+--------+----------+----------+
only showing top 20 rows



In [ ]:
result = schemaNGrams.groupBy("year", "ngram").agg(sum("ngramcount").alias("TotalPages"), sum("books").alias("TotalBooks")).orderBy("ngram")
result.show()


+----+--------+----------+----------+
|year|   ngram|TotalPages|TotalBooks|
+----+--------+----------+----------+
|1998|! $17.95|         4|         3|
|1992|! $17.95|         5|         5|
|1995|! $17.95|         1|         1|
|1997|! $17.95|         6|         5|
|1993|! $17.95|         2|         2|
|1987|! $17.95|         1|         1|
|1985|! $17.95|         1|         1|
|1991|! $17.95|         1|         1|
|1990|! $17.95|         1|         1|
|2007|! $17.95|         2|         2|
|2004|! $17.95|        14|        14|
|1996|! $17.95|         4|         2|
|2005|! $17.95|        13|        13|
|2001|! $17.95|         5|         4|
|2003|! $17.95|         2|         2|
|1999|! $17.95|        11|        10|
|2006|! $17.95|         5|         5|
|2008|! $17.95|         2|         2|
|2000|! $17.95|        11|         9|
|2002|! $17.95|         5|         5|
+----+--------+----------+----------+
only showing top 20 rows



Retourner le nombre total de bi-grammes différents dans chaque année, trié par ordre décroissant de l'année

In [ ]:
sqlContext.sql("SELECT year, COUNT(DISTINCT ngram) AS distinct_bigrams FROM ngrams GROUP BY year ORDER BY year DESC").show()


+----+----------------+
|year|distinct_bigrams|
+----+----------------+
|2008|               2|
|2007|               2|
|2006|               2|
|2005|               2|
|2004|               2|
|2003|               2|
|2002|               2|
|2001|               2|
|2000|               2|
|1999|               2|
|1998|               2|
|1997|               2|
|1996|               2|
|1995|               2|
|1993|               2|
|1992|               2|
|1991|               2|
|1990|               2|
|1987|               2|
|1985|               2|
+----+----------------+
only showing top 20 rows



In [ ]:
result = schemaNGrams.groupBy("year").agg(countDistinct("ngram")).orderBy("year", ascending=False)
result.show()


+----+---------------------+
|year|count(DISTINCT ngram)|
+----+---------------------+
|2008|                    2|
|2007|                    2|
|2006|                    2|
|2005|                    2|
|2004|                    2|
|2003|                    2|
|2002|                    2|
|2001|                    2|
|2000|                    2|
|1999|                    2|
|1998|                    2|
|1997|                    2|
|1996|                    2|
|1995|                    2|
|1993|                    2|
|1992|                    2|
|1991|                    2|
|1990|                    2|
|1987|                    2|
|1985|                    2|
+----+---------------------+
only showing top 20 rows

